In [ ]:
!pip install -U langchain
!pip install -U langchain-ollama
!pip install -U langchain-community
!pip install -U chromadb
!pip install -U sentence-transformers tqdm
!pip install -U ragas
!pip install -U datasets
!pip install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 135.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 100.0 kB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67

In [ ]:
import os
import torch
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TextSplitter,
    TokenTextSplitter,
)
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate
import pandas as pd
import time
import logging
from datasets import Dataset

# Define the directory containing the text file
current_dir = os.path.dirname('./')
file_path = os.path.join(current_dir, "hmao_npa.txt")
db_dir = os.path.join(current_dir, "db")

# Check if the text file exists
if not os.path.exists(file_path):
    raise FileNotFoundError(
        f"The file {file_path} does not exist. Please check the path."
    )

# Read the text content from the file
loader = TextLoader(file_path)
documents = loader.load()

# Define the embedding model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embeddings = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru",
    model_kwargs={'device': device}
)
store_name = "chroma_db_double_split"
persistent_directory = os.path.join(db_dir, store_name)

rec_char_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=100)
docs = rec_char_splitter.split_documents(documents)

print(f"Number of chunks after recursive splitting: {len(docs)}")
print("Creating Store")

db = Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory
        )

<ipython-input-3-c1dce2a432a9>:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secr

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Number of chunks after recursive splitting: 7729
Creating Store


In [ ]:
def query_vector_store(
    store_name, query, embedding_function, search_type, search_kwargs
):
    if os.path.exists(persistent_directory):
        print(f"\n--- Querying the Vector Store {store_name} ---")
        db = Chroma(
            persist_directory=persistent_directory,
            embedding_function=embedding_function,
        )
        retriever = db.as_retriever(
            search_type=search_type,
            search_kwargs=search_kwargs,
        )
        relevant_docs = retriever.invoke(query)
        # Display the relevant results with metadata
        print(f"\n--- Relevant Documents for {store_name} ---")
        for i, doc in enumerate(relevant_docs, 1):
            print(f"Document {i}:\n{doc.page_content}\n")
            if doc.metadata:
                print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")
    else:
        print(f"Vector store {store_name} does not exist.")



query = "Как зарплата влияет на мотивацию педагогов?"

print("\n--- Using Max Marginal Relevance (MMR) ---")
query_vector_store(
    store_name,
    query,
    embeddings,
    "similarity",
    {"k": 3},
)


--- Using Max Marginal Relevance (MMR) ---

--- Querying the Vector Store chroma_db_double_split ---

--- Relevant Documents for chroma_db_double_split ---
Document 1:
по формуле: (ЗПпнспо / ЗПнн) * 100, где: ЗПпнспо = {(ФЗПнспо / ЧСПпнспо) / 12} * 1000 ЗПпнспо - среднемесячная заработная плата преподавателей и мастеров производственного обучения списочного состава (без внешних совместителей) профессиональных образовательных организаций; ФЗПпнспо - фонд начисленной заработной платы преподавателей и мастеров производственного обучения списочного состава (без внешних совместителей) профессиональных образовательных организаций, всего (периодическая отчетность, форма N ЗП-образование); ЧСПпнспо - средняя численность преподавателей и мастеров производственного обучения списочного состава (без внешних совместителей) профессиональных образовательных организаций (без внешних совместителей) (периодическая отчетность, форма N ЗП-образование); ЗПнн - среднемесячный доход от трудовой деятельности

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])
ollama()
# ollama_thread = threading.Thread(target=ollama)
# ollama_thread.start()

from IPython.display import clear_output
!ollama pull owl/t-lite
clear_output()
import time

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import HumanMessage, SystemMessage

query = "Как зарплата влияет на мотивацию педагогов?"
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
relevant_docs = retriever.invoke(query)

# request(localhost.com)

print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")

combined_input = (
    "Вот несколько документов, которые могут помочь ответить на вопрос: "
    + query
    + "\n\nСоответствующие документы:\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\nПожалуйста, предоставьте ответ, основываясь только на предоставленных документах. Если ответ не найден в документах, ответьте 'Я не уверен'."
)

model = OllamaLLM(model="owl/t-lite")

messages = [
    SystemMessage(content="Ты ассистент отвечающий на вопросы по информации которая есть в нормативно-правовых-актах"),
    HumanMessage(content=combined_input),
]

# Invoke the model with the combined input
result = model.invoke(messages)

# Display the full result and content only
print("\n--- Generated Response ---")
# print("Full result:")
# print(result)
print("Content only:")
print(result)


--- Relevant Documents ---
Document 1:
по формуле: (ЗПпнспо / ЗПнн) * 100, где: ЗПпнспо = {(ФЗПнспо / ЧСПпнспо) / 12} * 1000 ЗПпнспо - среднемесячная заработная плата преподавателей и мастеров производственного обучения списочного состава (без внешних совместителей) профессиональных образовательных организаций; ФЗПпнспо - фонд начисленной заработной платы преподавателей и мастеров производственного обучения списочного состава (без внешних совместителей) профессиональных образовательных организаций, всего (периодическая отчетность, форма N ЗП-образование); ЧСПпнспо - средняя численность преподавателей и мастеров производственного обучения списочного состава (без внешних совместителей) профессиональных образовательных организаций (без внешних совместителей) (периодическая отчетность, форма N ЗП-образование); ЗПнн - среднемесячный доход от трудовой деятельности по автономному округу. Данные рассчитывает Депэкономики Югры, окончательные итоги - Росстат. 6. Отношение среднемесячной заработ

In [ ]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="owl/t-lite", num_gpu=1)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import HumanMessage, SystemMessage

query = "Как зарплата влияет на мотивацию педагогов?"
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
relevant_docs = retriever.invoke(query)

# request(localhost.com)

print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")

combined_input = (
    "Вот несколько документов, которые могут помочь ответить на вопрос: "
    + query
    + "\n\nСоответствующие документы:\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\nПожалуйста, предоставьте ответ, основываясь только на предоставленных документах. Если ответ не найден в документах, ответьте 'Я не уверен'."
)

model = OllamaLLM(model="owl/t-lite")

messages = [
    SystemMessage(content="Ты ассистент отвечающий на вопросы по информации которая есть в нормативно-правовых-актах"),
    HumanMessage(content=combined_input),
]

# Invoke the model with the combined input
result = model.invoke(messages)

# Display the full result and content only
print("\n--- Generated Response ---")
# print("Full result:")
# print(result)
print("Content only:")
print(result)

In [ ]:
class Evaluator:
    """
    Class for evaluating pipeline on test set.
    """
    def __init__(self, test_set_path: str):
        self.test_data = pd.read_excel(test_set_path).drop(columns=['contexts', 'evolution_type', 'metadata', 'episode_done'])

    def evaluate_pipe(self, pipeline) -> None:
        """
        Evaluate pipeline on test set.
        """
        answers = []
        contexts = []
        times = []
        for item in tqdm(self.test_data['question'].values):
            start_time = time.time()
            result = pipeline(item)
            end_time = time.time()
            answers.append(result['answer'])
            contexts.append(result['context'])
            times.append(end_time - start_time)

        mean_time = sum(times) / len(times)
        logging.info(f"Mean time for getting answer: {mean_time}")

        test_answers = self.test_data
        test_answers['answer'] = answers
        test_answers['contexts'] = contexts
        test_answers.to_csv("answers.csv")
        dataset = Dataset.from_dict(test_answers.to_dict(orient='list'))

        result = evaluate(
            dataset,
            metrics=[
                context_precision,
                faithfulness,
                answer_relevancy,
                context_recall,
            ],
        )
        df = result.to_pandas()
        df['mean_time'] = mean_time
        df.to_csv(f'results_{pipeline.__class__.__name__}.csv')
        logging.info(f"Saved results to 'results_{pipeline.__class__.__name__}.csv'")
        logging.info(f"Metrics: {result}")

In [ ]:
evaler = Evaluator("./v2_ragas_npa_dataset_firstPart.xlsx")

In [ ]:
def get_answer(question):
  retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
  )
  relevant_docs = retriever.invoke(question)

  # request(localhost.com)

  # print("\n--- Relevant Documents ---")
  cont = []
  for i, doc in enumerate(relevant_docs, 1):
      cont.append(doc.page_content)

  combined_input = (
      "Вот несколько документов, которые могут помочь ответить на вопрос: "
      + query
      + "\n\nСоответствующие документы:\n"
      + "\n\n".join([doc.page_content for doc in relevant_docs])
      + "\n\nПожалуйста, предоставьте ответ, основываясь только на предоставленных документах. Если ответ не найден в документах, ответьте 'Я не уверен'."
  )

  model = OllamaLLM(model="owl/t-lite")

  messages = [
      SystemMessage(content="Ты ассистент отвечающий на вопросы по информации которая есть в нормативно-правовых-актах"),
      HumanMessage(content=combined_input),
  ]

  # Invoke the model with the combined input
  result = model.invoke(messages)
  return {"answer": result, "context": cont}

In [ ]:
get_answer("Как зарплата влияет на мотивацию педагогов?")

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = ""

In [ ]:
evaler.evaluate_pipe(get_answer)